# San Francisco Housing Cost Analysis

In this assignment, you will perform fundamental analysis for the San Francisco housing market to allow potential real estate investors to choose rental investment properties.

In [141]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
from dotenv import load_dotenv

import warnings
warnings.filterwarnings('ignore')

In [142]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")

## Load Data

In [143]:
# Read the census data into a Pandas DataFrame
file_path = Path("../Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")
sfo_data.head()

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
year,,,,
2010,Alamo Square,291.182945,372560,1239
2010,Anza Vista,267.932583,372560,1239
2010,Bayview,170.098665,372560,1239
2010,Buena Vista Park,347.394919,372560,1239
2010,Central Richmond,319.027623,372560,1239


- - - 

## Housing Units Per Year

In this section, you will calculate the number of housing units per year and visualize the results as a bar chart using the Pandas plot function.

**Hint:** Use the Pandas `groupby` function.

**Optional challenge:** Use the min, max, and std to scale the y limits of the chart.



In [164]:
# Calculate the mean number of housing units per year (hint: use groupby) 
year_grouping_house = sfo_data.groupby('year')[['housing_units']].mean()
year_grouping_house

,housing_units
year,
2010,372560
2011,374507
2012,376454
2013,378401
2014,380348
2015,382295
2016,384242


In [165]:
# Save the dataframe as a csv file
year_grouping_house.to_csv(Path('../Data/year_grouping_house.csv'))

In [146]:
# Use the Pandas plot function to plot the average housing units per year.
# Note: You will need to manually adjust the y limit of the chart using the min and max values from above.
# Optional Challenge: Use the min, max, and std to scale the y limits of the chart
year_grouping_house.hvplot.bar(ylim=[year_grouping_house.min() - year_grouping_house.std(), year_grouping_house.max() + year_grouping_house.std()], height=600, title='Housing Units Per Year', color='purple').opts(bgcolor='darkgray')

:Bars   [year]   (housing_units)

- - - 

## Average Housing Costs in San Francisco Per Year

In this section, you will calculate the average monthly rent and the average price per square foot for each year. An investor may wish to better understand the sales price of the rental property over time. For example, a customer will want to know if they should expect an increase or decrease in the property value over time so they can determine how long to hold the rental property.  Plot the results as two line charts.

**Optional challenge:** Plot each line chart in a different color.

In [147]:
# Calculate the average sale price per square foot and average gross rent
year_grouping = sfo_data.groupby('year')[['sale_price_sqr_foot', 'gross_rent']].mean()
year_grouping.to_csv(Path('../Data/year_grouping.csv'))

In [163]:
# Create two line charts, one to plot the average sale price per square foot and another for average montly rent

# Line chart for average sale price per square foot               # Line chart for average montly rent
year_grouping['sale_price_sqr_foot'].hvplot.line(x='year', y='sale_price_sqr_foot', color="green", title='Average Sale Price Per Square Foot').opts(bgcolor='lightgray') + year_grouping['gross_rent'].hvplot.line(x='year', y='gross_rent', color='blue', title='Average Monthly Rent').opts(bgcolor='lightgray')

:Layout
   .Curve.Sale_price_sqr_foot :Curve   [year]   (sale_price_sqr_foot)
   .Curve.Gross_rent          :Curve   [year]   (gross_rent)

- - - 

## Average Prices by Neighborhood

In this section, you will use hvplot to create two interactive visulizations of average prices with a dropdown selector for the neighborhood. The first visualization will be a line plot showing the trend of average price per square foot over time for each neighborhood.  The second will be a line plot showing the trend of average montly rent over time for each neighborhood.

**Hint:** It will be easier to create a new DataFrame from grouping the data and calculating the mean prices for each year and neighborhood

In [149]:
# Group by year and neighborhood and then create a new dataframe of the mean values
new_sfo_df = pd.DataFrame(
    sfo_data.groupby(
        ['year', 'neighborhood'])[['sale_price_sqr_foot', 'housing_units', 'gross_rent']].mean()
        ).reset_index()

new_sfo_df.to_csv(Path('../Data/new_sfo_df.csv'))
new_sfo_df[new_sfo_df['neighborhood'] == 'Alamo Square']

,year,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
0,2010,Alamo Square,291.182945,372560,1239
59,2011,Alamo Square,272.527310,374507,1530
118,2012,Alamo Square,183.099317,376454,2324
176,2013,Alamo Square,387.794144,378401,2971
233,2014,Alamo Square,484.443552,380348,3528
293,2015,Alamo Square,602.115877,382295,3739
351,2016,Alamo Square,340.981835,384242,4390


In [150]:
# Use hvplot to create an interactive line chart of the average price per sq ft.
# The plot should have a dropdown selector for the neighborhood
new_sfo_df.hvplot.line(x='year', y='sale_price_sqr_foot', groupby='neighborhood', ylabel='Sale Price Per Square Foot', color='darkblue').opts(bgcolor='lightgray')

:DynamicMap   [neighborhood]
   :Curve   [year]   (sale_price_sqr_foot)

In [151]:
# Use hvplot to create an interactive line chart of the average monthly rent.
# The plot should have a dropdown selector for the neighborhood
new_sfo_df.hvplot.line(x='year', y='gross_rent', groupby='neighborhood', ylabel='Gross Rent').opts(bgcolor='lightgray')

:DynamicMap   [neighborhood]
   :Curve   [year]   (gross_rent)

## The Top 10 Most Expensive Neighborhoods

In this section, you will need to calculate the mean sale price per square foot for each neighborhood and then sort the values to obtain the top 10 most expensive neighborhoods on average. Plot the results as a bar chart.

In [152]:
# Getting the data from the top 10 expensive neighborhoods to own

top_10_neighborhood = new_sfo_df.groupby(['neighborhood']).mean().sort_values('sale_price_sqr_foot', ascending=False).head(10)
top_10_list = top_10_neighborhood.index.values
top_10_neighborhood.to_csv(Path('../Data/top_10_neighborhood.csv'))
top_10_neighborhood.hvplot.bar(y='gross_rent', rot=90, height=500, ylabel='Gross Rent', xlabel='Neighborhood', title='Top 10 Expensive Neighborhoods')

:Bars   [neighborhood]   (gross_rent)

In [153]:
# Plotting the data from the top 10 expensive neighborhoods
top_10_neighborhood_by_year = new_sfo_df[new_sfo_df['neighborhood'].isin(top_10_list)].round(2)
top_10_neighborhood_by_year.to_csv(Path('../Data/top_10_neighborhood_by_year.csv'))
top_10_neighborhood_by_year.hvplot.bar(x='year', y=['sale_price_sqr_foot', 'gross_rent'], groupby='neighborhood', rot=70, xlabel="Year", height=600, title='Comparison of Rent and Sales Prices of Most Expensive Neighborhoods')

:DynamicMap   [neighborhood]
   :Bars   [year,Variable]   (value)

- - - 

## Comparing cost to purchase versus rental income

In this section, you will use `hvplot` to create an interactive visualization with a dropdown selector for the neighborhood. This visualization will feature a side-by-side comparison of average price per square foot versus average montly rent by year.  

**Hint:** Use the `hvplot` parameter, `groupby`, to create a dropdown selector for the neighborhood.

In [154]:
# Fetch the previously generated DataFrame that was grouped by year and neighborhood
new_sfo_df.head()

,year,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
0,2010,Alamo Square,291.182945,372560,1239
1,2010,Anza Vista,267.932583,372560,1239
2,2010,Bayview,170.098665,372560,1239
3,2010,Buena Vista Park,347.394919,372560,1239
4,2010,Central Richmond,319.027623,372560,1239


- - - 

## Neighborhood Map

In this section, you will read in neighborhoods location data and build an interactive map with the average house value per neighborhood. Use a `scatter_mapbox` from Plotly express to create the visualization. Remember, you will need your Mapbox API key for this.

### Load Location Data

In [155]:
# Load neighborhoods coordinates data
neighborhood_coordinates = pd.read_csv(Path('../Data/neighborhoods_coordinates.csv')).rename(columns={'Neighborhood': 'neighborhood'}).set_index('neighborhood')
neighborhood_coordinates.head()

,Lat,Lon
neighborhood,,
Alamo Square,37.791012,-122.402100
Anza Vista,37.779598,-122.443451
Bayview,37.734670,-122.401060
Bayview Heights,37.728740,-122.410980
Bernal Heights,37.728630,-122.443050


### Data Preparation

You will need to join the location data with the mean values per neighborhood.

1. Calculate the mean values for each neighborhood.

2. Join the average values with the neighborhood locations.

In [156]:
# Calculate the mean values for each neighborhood
neighborhood_mean = new_sfo_df.groupby('neighborhood').mean()
neighborhood_mean.head()

,year,sale_price_sqr_foot,housing_units,gross_rent
neighborhood,,,,
Alamo Square,2013.000000,366.020712,378401.0,2817.285714
Anza Vista,2013.333333,373.382198,379050.0,3031.833333
Bayview,2012.000000,204.588623,376454.0,2318.400000
Bayview Heights,2015.000000,590.792839,382295.0,3739.000000
Bernal Heights,2013.500000,576.746488,379374.5,3080.333333


In [157]:
# Join neighborhood means dataframe with neighborhood coordinates dataframe
neighborhood_geolocation = pd.concat([neighborhood_mean, neighborhood_coordinates], join='inner', axis=1)
neighborhood_geolocation.to_csv(Path('../Data/neighborhood_geolocation.csv'))
neighborhood_geolocation.head()

,year,sale_price_sqr_foot,housing_units,gross_rent,Lat,Lon
neighborhood,,,,,,
Alamo Square,2013.000000,366.020712,378401.0,2817.285714,37.791012,-122.402100
Anza Vista,2013.333333,373.382198,379050.0,3031.833333,37.779598,-122.443451
Bayview,2012.000000,204.588623,376454.0,2318.400000,37.734670,-122.401060
Bayview Heights,2015.000000,590.792839,382295.0,3739.000000,37.728740,-122.410980
Buena Vista Park,2012.833333,452.680591,378076.5,2698.833333,37.768160,-122.439330


### Mapbox Visualization

Plot the average values per neighborhood using a Plotly express `scatter_mapbox` visualization.

In [158]:
# Set the mapbox access token
px.set_mapbox_access_token(map_box_api)

# Round Units
neighborhood_geolocation = neighborhood_geolocation.round(2)

# Create a scatter mapbox to analyze neighborhood info
map_plot = px.scatter_mapbox(
    neighborhood_geolocation,
    lat="Lat",
    lon="Lon",
    size='sale_price_sqr_foot',
    color=neighborhood_geolocation.index,
    zoom=10
)

- - -

## Cost Analysis - Optional Challenge

In this section, you will use Plotly express to create visualizations that investors can use to interactively filter and explore various factors related to the house value of the San Francisco's neighborhoods. 

### Create a DataFrame showing the most expensive neighborhoods in San Francisco by year

In [159]:
# Fetch the data from all expensive neighborhoods per year.
top_10_neighborhood = top_10_neighborhood.reset_index()

# Round Data
top_10_neighborhood = top_10_neighborhood.round(2)

top_10_neighborhood

,neighborhood,year,sale_price_sqr_foot,housing_units,gross_rent
0,Union Square District,2012.50,903.99,377427.50,2555.17
1,Merced Heights,2014.00,788.84,380348.00,3414.00
2,Miraloma Park,2011.75,779.81,375967.25,2155.25
3,Pacific Heights,2013.00,689.56,378401.00,2817.29
4,Westwood Park,2015.00,687.09,382295.00,3959.00
5,Telegraph Hill,2013.00,676.51,378401.00,2817.29
6,Presidio Heights,2013.00,675.35,378401.00,2817.29
7,Cow Hollow,2013.00,665.96,378401.00,2817.29
8,Potrero Hill,2013.00,662.01,378401.00,2817.29
9,South Beach,2011.67,650.12,375805.00,2099.00


### Create a parallel coordinates plot and parallel categories plot of most expensive neighborhoods in San Francisco per year


In [160]:
# Parallel Categories Plot

px.parallel_categories(
    top_10_neighborhood,
    dimensions=['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent'],
    color_continuous_scale=px.colors.sequential.Inferno,
    color='sale_price_sqr_foot'
    )

In [161]:
# Parallel Coordinates Plot
px.parallel_coordinates(
    top_10_neighborhood,
    dimensions=['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent'],
    color_continuous_scale=px.colors.sequential.Inferno,
    color='gross_rent'
)

### Create a sunburst chart to conduct a costs analysis of most expensive neighborhoods in San Francisco per year

In [166]:
# Sunburst Plot
px.sunburst(
    top_10_neighborhood_by_year,
    path=['year', 'neighborhood'],
    values='gross_rent'
)

AttributeError: type object 'object' has no attribute 'dtype'